In [1]:
import pandas as pd
import numpy as np
from fbprophet import Prophet
import tensorflow as tf
import matplotlib
import os
import matplotlib.pyplot as plt
from datetime import datetime
tf.set_random_seed(77)

mockForecastDictionary = {}
realForecastDictionary = {}

def LearningModuleRunner(rawArrayDatas, processId, forecastDay,dayOrWeekOrMonth):
    #TODO make dayOrWeekOrMonth parameter
    dayOrWeekOrMonth=dayOrWeekOrMonth
    # options:
    # 'day', 'week', 'month'

    feature = 'DayOfWeek_WeekNumber_Month_Season'
    # options:
    # dayOrWeekOrMonth='day': 'DayOfWeek_WeekNumber_Month_Season','DayOfWeek01_WeekNumber_Month_Season'//
    # dayOrWeekOrMonth='week': 'WeekNumber_Month_Season_Year'

#     LoggingManager.PrintLogMessage("LearningManager", "LearningModuleRunner", "start of learning #" + str(processId), DefineManager.LOG_LEVEL_INFO)

    global mockForecastDictionary
    global realForecastDictionary
    mockForcastDay=forecastDay

    ##Make txsForRealForecastLstm   [:]
    ds = rawArrayDatas[0]
    y = list((rawArrayDatas[1]))
    sales = list(zip(ds, y))
    txsForRealForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastLstm [:-forecastDay]
    ds = rawArrayDatas[0][:-forecastDay]
    y= list((rawArrayDatas[1][:-forecastDay] ))
    sales = list(zip(ds, y))
    txsForMockForecastLstm =pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForRealForecastBayesian [:-forecastDay] & np.log
    ds = rawArrayDatas[0][:-forecastDay]
    # TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y = list(np.log(rawArrayDatas[1][:-forecastDay]))
    sales = list(zip(ds, y))
    txsForRealForecastBayesian = pd.DataFrame(data=sales, columns=['ds', 'y'])

    ##Make txsForMockForecastBayseian   [:-(mockForcastDay+forecastDay)] & np.log
    ds = rawArrayDatas[0][:-(mockForcastDay+forecastDay)]
    #TODO bayseian에 대해서는 input값이 0인 상황처리 필요
    y= list(np.log(rawArrayDatas[1][:-(mockForcastDay+forecastDay)]))
    sales = list(zip(ds, y))
    txsForMockForecastBayseian =pd.DataFrame(data=sales, columns=['ds', 'y'])

    #testY for algorithm compare has size of (mockForcastDay+forecastDay)  rawArrayDatas[1][-(mockForcastDay+forecastDay):-forecastDay]
    testY= rawArrayDatas[1][-(mockForcastDay+forecastDay):-forecastDay]


    if dayOrWeekOrMonth is 'day':
        ####LSTM_day

        #select feature module
        feature='DayOfWeek_WeekNumber_Month_Season'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay,feature)

      
        ####Bayseian_day

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'day')

        #알고리즘 비교
        nameOfBestAlgorithm= AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출       
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'day')
            
    elif dayOrWeekOrMonth is 'week':
        
        ####LSTM_week

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)


        ####Bayseian_week

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'week')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'week')
    
            
    elif dayOrWeekOrMonth is 'month':
       
        ####LSTM_month

        # select feature module
        feature = 'WeekNumber_Month_Season_Year'

        mockForecastDictionary['LSTM'] = LSTM(txsForMockForecastLstm, mockForcastDay, feature)    

        ####Bayseian_month

        mockForecastDictionary['Bayseian'] = Bayseian(txsForMockForecastBayseian, mockForcastDay, 'month')


        # 알고리즘 비교
        nameOfBestAlgorithm = AlgorithmCompare(testY)
        
        ####더 좋은 알고리즘 호출
        tf.reset_default_graph()
        realForecastDictionary['LSTM'] = LSTM(txsForRealForecastLstm, forecastDay,feature)
        realForecastDictionary['Bayseian'] = Bayseian(txsForRealForecastBayesian, forecastDay, 'month')

        ####################################################################################BAYSEIAN
    print('LSTM realforecast :',realForecastDictionary['LSTM'])
    print('Bayseian realforecast :',realForecastDictionary['Bayseian'] ) 
    data = rawArrayDatas[1][:-forecastDay] + realForecastDictionary[nameOfBestAlgorithm]
    date= rawArrayDatas[0]
    print("===================================THE END===================================================")
    return realForecastDictionary[nameOfBestAlgorithm]


def LSTM(txs, forecastDay, features):
    tf.reset_default_graph()
    tf.set_random_seed(77)
    # Add basic date related features to the table
    year = lambda x: datetime.strptime(x, "%Y-%m-%d").year
    dayOfWeek = lambda x: datetime.strptime(x, "%Y-%m-%d").weekday()
    month = lambda x: datetime.strptime(x, "%Y-%m-%d").month
    weekNumber = lambda x: datetime.strptime(x, "%Y-%m-%d").strftime('%V')
    txs['year'] = txs['ds'].map(year)
    txs['month'] = txs['ds'].map(month)
    txs['weekNumber'] = txs['ds'].map(weekNumber)
    txs['dayOfWeek'] = txs['ds'].map(dayOfWeek)

    # Add non-basic date related features to the table
    seasons = [0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0]  # dec - feb is winter, then spring, summer, fall etc
    season = lambda x: seasons[(datetime.strptime(x, "%Y-%m-%d").month - 1)]
    day_of_week01s = [0, 0, 0, 0, 0, 1, 1]
    day_of_week01 = lambda x: day_of_week01s[(datetime.strptime(x, "%Y-%m-%d").weekday())]
    txs['season'] = txs['ds'].map(season)
    txs['dayOfWeek01'] = txs['ds'].map(day_of_week01)

    # Backup originalSales
    originalSales = list(txs['y'])
    sales = list(txs['y'])
    
    #week number는 경계부분에서 약간 오류가 있다.
    if features is 'DayOfWeek_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]
    elif features is 'DayOfWeek01_WeekNumber_Month_Season':
        tempxy = [list(txs['dayOfWeek01']), list(txs['weekNumber']), list(txs['month']), list(txs['season']), sales]

    elif features is 'WeekNumber_Month_Season_Year':
        tempxy = [list(txs['weekNumber']), list(txs['month']), list(txs['season']), list(txs['year']), sales]
    
   
    xy = np.array(tempxy).transpose().astype(np.float)
    


    # Backup originalXY for denormalize
    originalXY = np.array(tempxy).transpose().astype(np.float)
    xy = minMaxNormalizer(xy)

    # TRAIN PARAMETERS
    # data_dim은 y값 도출을 위한 feature 가지수+1(독립변수 가지수 +1(y포함))
    data_dim = 5
    # data_dim크기의 data 한 묶음이 seq_length만큼 input으로 들어가
    seq_length = 10
    # output_dim(=forecastDays)만큼의 다음날 y_data를 예측
    output_dim = forecastDay
    # hidden_dim은 정말 임의로 설정
    hidden_dim = 100
    # learning rate은 배우는 속도(너무 크지도, 작지도 않게 설정)
    learning_rate = 0.001
    iterations=1000
    # Build a series dataset(seq_length에 해당하는 전날 X와 다음 forecastDays에 해당하는 Y)
    x = xy
    y = xy[:, [-1]]
    dataX = []
    dataY = []
    for i in range(0, len(y) - seq_length - forecastDay + 1):
        _x = x[i:i + seq_length]
        _y = y[i + seq_length:i + seq_length + forecastDay]
        _y = np.reshape(_y, (forecastDay))
        dataX.append(_x)
        dataY.append(_y)

    train_size = int(len(dataY) - forecastDay)
    # train_size = int(len(dataY) * 0.7)
    test_size = len(dataY) - train_size

    trainX, testX = np.array(dataX[0:train_size]), np.array(dataX[train_size:])

    trainY, testY = np.array(dataY[0:train_size]), np.array(dataY[train_size:])

    X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
    Y = tf.placeholder(tf.float32, [None, forecastDay])

    cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
    outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
    Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)
    loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
    optimizer = tf.train.AdamOptimizer(learning_rate)
    train = optimizer.minimize(loss)

    denormalizedTestY = originalSales[train_size + seq_length:]
    #     denormalizedTestY_feed=np.array([[i] for i in denormalizedTestY])

    targets = tf.placeholder(tf.float32, [None, 1])
    predictions = tf.placeholder(tf.float32, [None, 1])

    count = 0
    with tf.Session() as sess:

        # 초기화
        init = tf.global_variables_initializer()
        sess.run(init)

        # Training step
        for i in range(iterations):

            _, step_loss = sess.run([train, loss], feed_dict={X: trainX, Y: trainY})
            print("[step: {}] loss: {}".format(i, step_loss))


        # Test step
        test_predict = minMaxDeNormalizer(sess.run(Y_pred, feed_dict={X: testX}), originalXY)
        realSale = minMaxDeNormalizer(testY[-1], originalXY)
#         listedLogPredict=test_predict[-1].tolist()
#     return [np.exp(y) for y in listedLogPredict]
    return test_predict[-1].tolist()

def Bayseian(txs, forecastDay, unit):
    global mockForecastDictionary
    global realForecastDictionary

    if unit is 'day':
        if (len(txs) < 366):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay)
            forecastProphetTable = model.predict(future)


    elif unit is 'week':
        if(len(txs)<53):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='w')
            forecastProphetTable = model.predict(future)

    elif unit is 'month':
        if(len(txs)<12):
            model = Prophet()
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)

        else:
            model = Prophet(yearly_seasonality=True)
            model.fit(txs)
            future = model.make_future_dataframe(periods=forecastDay,freq='m')
            forecastProphetTable = model.predict(future)


    #date = [d.strftime('%Y-%m-%d') for d in forecastProphetTable['ds']]
    return [np.exp(y) for y in forecastProphetTable['yhat'][-forecastDay:]]

def rmse(a,b):
    sum=0
    for i in range(len(a)):
        sum=sum+(a[i]-b[i])**2
    return np.sqrt(sum/len(a))

def minMaxNormalizer(data):
    numerator=data-np.min(data)
    denominator=np.max(data)-np.min(data)
    return numerator/(denominator+1e-7)

def minMaxDeNormalizer(data, originalData):
    shift=np.min(originalData)
    multiplier=np.max(originalData)-np.min(originalData)
    return (data+shift)*multiplier

def AlgorithmCompare(testY):
    global mockForecastDictionary
    nameOfBestAlgorithm = 'LSTM'
    minData = rmse(testY, mockForecastDictionary[nameOfBestAlgorithm])
    rms = 0
    for algorithm in mockForecastDictionary.keys():
        rms = rmse(testY, mockForecastDictionary[algorithm])
        if rms < minData:
            nameOfBestAlgorithm = algorithm
    print('testY is: ', testY)
    print('\n')
    print('LSTM testforecast :',mockForecastDictionary['LSTM'] , '\n@@@@@LSTM rmse: ',rmse(testY, mockForecastDictionary['LSTM']) )
    print('Bayseian testforecast :',mockForecastDictionary['Bayseian'], '\n@@@@@Bayseian rmse: ',rmse(testY, mockForecastDictionary['Bayseian']) ) 
    print('\n')
    print(nameOfBestAlgorithm, 'WON!!!!!!')
    return nameOfBestAlgorithm


In [2]:
# def LSTM(txs, forecastDay, features)
# def Bayseian(txs, forecastDay, unit)

In [6]:
columns=['ds','y']
df=pd.read_table('walWeek.csv', sep=',',header=None,names=columns )
forecastDay=int(len(df)*0.2)
print(len(df), forecastDay)


143 28


In [7]:
df['y'][-forecastDay:]=[1]*forecastDay
rawArrayDatas=[[i for i in df['ds']],[i for i in df['y']]]

C:\Users\User\Anaconda3\envs\tensorflow35\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
print('{"Date":',rawArrayDatas[0],',"Data":',rawArrayDatas[1],',"Day":',forecastDay,'}')

{"Date": ['2010-02-05', '2010-02-12', '2010-02-19', '2010-02-26', '2010-03-05', '2010-03-12', '2010-03-19', '2010-03-26', '2010-04-02', '2010-04-09', '2010-04-16', '2010-04-23', '2010-04-30', '2010-05-07', '2010-05-14', '2010-05-21', '2010-05-28', '2010-06-04', '2010-06-11', '2010-06-18', '2010-06-25', '2010-07-02', '2010-07-09', '2010-07-16', '2010-07-23', '2010-07-30', '2010-08-06', '2010-08-13', '2010-08-20', '2010-08-27', '2010-09-03', '2010-09-10', '2010-09-17', '2010-09-24', '2010-10-01', '2010-10-08', '2010-10-15', '2010-10-22', '2010-10-29', '2010-11-05', '2010-11-12', '2010-11-19', '2010-11-26', '2010-12-03', '2010-12-10', '2010-12-17', '2010-12-24', '2010-12-31', '2011-01-07', '2011-01-14', '2011-01-21', '2011-01-28', '2011-02-04', '2011-02-11', '2011-02-18', '2011-02-25', '2011-03-04', '2011-03-11', '2011-03-18', '2011-03-25', '2011-04-01', '2011-04-08', '2011-04-15', '2011-04-22', '2011-04-29', '2011-05-06', '2011-05-13', '2011-05-20', '2011-05-27', '2011-06-03', '2011-06-1

In [9]:
answer=LearningModuleRunner(rawArrayDatas,0,forecastDay,'week') #0은 processId

[step: 0] loss: 284.3252258300781
[step: 1] loss: 273.5332946777344
[step: 2] loss: 263.53515625
[step: 3] loss: 254.09426879882812
[step: 4] loss: 245.02047729492188
[step: 5] loss: 236.13613891601562
[step: 6] loss: 227.2740936279297
[step: 7] loss: 218.28106689453125
[step: 8] loss: 209.0157012939453
[step: 9] loss: 199.3433837890625
[step: 10] loss: 189.13375854492188
[step: 11] loss: 178.2645721435547
[step: 12] loss: 166.6344451904297
[step: 13] loss: 154.1919403076172
[step: 14] loss: 140.98886108398438
[step: 15] loss: 127.27042388916016
[step: 16] loss: 113.60594177246094
[step: 17] loss: 101.03141784667969
[step: 18] loss: 91.02660369873047
[step: 19] loss: 84.91893005371094
[step: 20] loss: 82.43571472167969
[step: 21] loss: 80.8990478515625
[step: 22] loss: 77.31671142578125
[step: 23] loss: 71.1046142578125
[step: 24] loss: 63.857017517089844
[step: 25] loss: 57.6027946472168
[step: 26] loss: 53.583900451660156
[step: 27] loss: 51.939292907714844
[step: 28] loss: 52.015914

[step: 259] loss: 37.03400421142578
[step: 260] loss: 37.01252746582031
[step: 261] loss: 36.991058349609375
[step: 262] loss: 36.96962356567383
[step: 263] loss: 36.94822692871094
[step: 264] loss: 36.926876068115234
[step: 265] loss: 36.90558624267578
[step: 266] loss: 36.884368896484375
[step: 267] loss: 36.86323547363281
[step: 268] loss: 36.8421745300293
[step: 269] loss: 36.82121276855469
[step: 270] loss: 36.80034637451172
[step: 271] loss: 36.779563903808594
[step: 272] loss: 36.758907318115234
[step: 273] loss: 36.73835372924805
[step: 274] loss: 36.7178955078125
[step: 275] loss: 36.697547912597656
[step: 276] loss: 36.67729568481445
[step: 277] loss: 36.65715026855469
[step: 278] loss: 36.6370735168457
[step: 279] loss: 36.617103576660156
[step: 280] loss: 36.59721374511719
[step: 281] loss: 36.57740020751953
[step: 282] loss: 36.55764389038086
[step: 283] loss: 36.53794860839844
[step: 284] loss: 36.518310546875
[step: 285] loss: 36.49871063232422
[step: 286] loss: 36.47916

[step: 522] loss: 31.59561538696289
[step: 523] loss: 31.627836227416992
[step: 524] loss: 31.629241943359375
[step: 525] loss: 31.514114379882812
[step: 526] loss: 31.540523529052734
[step: 527] loss: 31.513872146606445
[step: 528] loss: 31.43914031982422
[step: 529] loss: 31.441268920898438
[step: 530] loss: 31.41635513305664
[step: 531] loss: 31.35863494873047
[step: 532] loss: 31.339059829711914
[step: 533] loss: 31.327791213989258
[step: 534] loss: 31.271404266357422
[step: 535] loss: 31.240785598754883
[step: 536] loss: 31.237895965576172
[step: 537] loss: 31.182497024536133
[step: 538] loss: 31.147125244140625
[step: 539] loss: 31.141637802124023
[step: 540] loss: 31.09527015686035
[step: 541] loss: 31.056997299194336
[step: 542] loss: 31.039310455322266
[step: 543] loss: 31.00749397277832
[step: 544] loss: 30.97039794921875
[step: 545] loss: 30.93646240234375
[step: 546] loss: 30.912635803222656
[step: 547] loss: 30.884057998657227
[step: 548] loss: 30.841712951660156
[step: 54

[step: 750] loss: 22.725990295410156
[step: 751] loss: 22.84661102294922
[step: 752] loss: 22.84684181213379
[step: 753] loss: 22.835065841674805
[step: 754] loss: 22.639360427856445
[step: 755] loss: 22.448535919189453
[step: 756] loss: 22.250986099243164
[step: 757] loss: 22.12891387939453
[step: 758] loss: 22.081453323364258
[step: 759] loss: 22.093719482421875
[step: 760] loss: 22.148048400878906
[step: 761] loss: 22.219079971313477
[step: 762] loss: 22.321300506591797
[step: 763] loss: 22.37651252746582
[step: 764] loss: 22.44508934020996
[step: 765] loss: 22.373958587646484
[step: 766] loss: 22.28668212890625
[step: 767] loss: 22.097034454345703
[step: 768] loss: 21.93514633178711
[step: 769] loss: 21.800058364868164
[step: 770] loss: 21.721284866333008
[step: 771] loss: 21.68940544128418
[step: 772] loss: 21.6928653717041
[step: 773] loss: 21.726577758789062
[step: 774] loss: 21.785110473632812
[step: 775] loss: 21.89767074584961
[step: 776] loss: 22.027664184570312
[step: 777] 

[step: 975] loss: 18.037212371826172
[step: 976] loss: 18.800647735595703
[step: 977] loss: 18.57923126220703
[step: 978] loss: 18.553138732910156
[step: 979] loss: 18.29905891418457
[step: 980] loss: 17.758684158325195
[step: 981] loss: 17.060434341430664
[step: 982] loss: 17.367443084716797
[step: 983] loss: 17.64583969116211
[step: 984] loss: 17.557422637939453
[step: 985] loss: 17.864025115966797
[step: 986] loss: 17.573501586914062
[step: 987] loss: 17.124774932861328
[step: 988] loss: 17.118980407714844
[step: 989] loss: 17.025253295898438
[step: 990] loss: 17.00511932373047
[step: 991] loss: 17.25263214111328
[step: 992] loss: 17.31804847717285
[step: 993] loss: 17.210058212280273
[step: 994] loss: 17.216060638427734
[step: 995] loss: 16.965755462646484
[step: 996] loss: 16.84280776977539
[step: 997] loss: 16.844757080078125
[step: 998] loss: 16.76294708251953
[step: 999] loss: 16.799579620361328
Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override 

[step: 180] loss: 56.553977966308594
[step: 181] loss: 56.516578674316406
[step: 182] loss: 56.48052215576172
[step: 183] loss: 56.44593048095703
[step: 184] loss: 56.41276168823242
[step: 185] loss: 56.38093566894531
[step: 186] loss: 56.35038757324219
[step: 187] loss: 56.32088088989258
[step: 188] loss: 56.292259216308594
[step: 189] loss: 56.26428985595703
[step: 190] loss: 56.23686599731445
[step: 191] loss: 56.209896087646484
[step: 192] loss: 56.18334197998047
[step: 193] loss: 56.157203674316406
[step: 194] loss: 56.13148498535156
[step: 195] loss: 56.10627746582031
[step: 196] loss: 56.0816650390625
[step: 197] loss: 56.057708740234375
[step: 198] loss: 56.034385681152344
[step: 199] loss: 56.011756896972656
[step: 200] loss: 55.98982238769531
[step: 201] loss: 55.96845245361328
[step: 202] loss: 55.94769287109375
[step: 203] loss: 55.92745590209961
[step: 204] loss: 55.90769958496094
[step: 205] loss: 55.888328552246094
[step: 206] loss: 55.86933517456055
[step: 207] loss: 55

[step: 424] loss: 53.24851989746094
[step: 425] loss: 53.235931396484375
[step: 426] loss: 53.216644287109375
[step: 427] loss: 53.20585632324219
[step: 428] loss: 53.19492721557617
[step: 429] loss: 53.17727279663086
[step: 430] loss: 53.16357421875
[step: 431] loss: 53.1531867980957
[step: 432] loss: 53.13777160644531
[step: 433] loss: 53.12199783325195
[step: 434] loss: 53.11041259765625
[step: 435] loss: 53.09733581542969
[step: 436] loss: 53.081260681152344
[step: 437] loss: 53.06726837158203
[step: 438] loss: 53.054996490478516
[step: 439] loss: 53.040489196777344
[step: 440] loss: 53.02488708496094
[step: 441] loss: 53.0111198425293
[step: 442] loss: 52.99785614013672
[step: 443] loss: 52.98305130004883
[step: 444] loss: 52.96755599975586
[step: 445] loss: 52.953094482421875
[step: 446] loss: 52.93913269042969
[step: 447] loss: 52.92431640625
[step: 448] loss: 52.908687591552734
[step: 449] loss: 52.89323425292969
[step: 450] loss: 52.87830352783203
[step: 451] loss: 52.86328887

[step: 669] loss: 47.82582092285156
[step: 670] loss: 48.0982666015625
[step: 671] loss: 47.834625244140625
[step: 672] loss: 47.68763732910156
[step: 673] loss: 47.89250946044922
[step: 674] loss: 47.76814270019531
[step: 675] loss: 47.59241485595703
[step: 676] loss: 47.68463897705078
[step: 677] loss: 47.704376220703125
[step: 678] loss: 47.5701904296875
[step: 679] loss: 47.49617004394531
[step: 680] loss: 47.57002258300781
[step: 681] loss: 47.62236022949219
[step: 682] loss: 47.53455352783203
[step: 683] loss: 47.43849563598633
[step: 684] loss: 47.42301940917969
[step: 685] loss: 47.45448303222656
[step: 686] loss: 47.470149993896484
[step: 687] loss: 47.390872955322266
[step: 688] loss: 47.312721252441406
[step: 689] loss: 47.27191162109375
[step: 690] loss: 47.27581024169922
[step: 691] loss: 47.29023361206055
[step: 692] loss: 47.268089294433594
[step: 693] loss: 47.23090362548828
[step: 694] loss: 47.19575881958008
[step: 695] loss: 47.20429229736328
[step: 696] loss: 47.269

[step: 915] loss: 43.540550231933594
[step: 916] loss: 43.536285400390625
[step: 917] loss: 43.5157470703125
[step: 918] loss: 43.48530197143555
[step: 919] loss: 43.44404602050781
[step: 920] loss: 43.40334701538086
[step: 921] loss: 43.36402893066406
[step: 922] loss: 43.335899353027344
[step: 923] loss: 43.31918716430664
[step: 924] loss: 43.32695007324219
[step: 925] loss: 43.37199783325195
[step: 926] loss: 43.50353240966797
[step: 927] loss: 43.77412414550781
[step: 928] loss: 44.26663589477539
[step: 929] loss: 44.69652557373047
[step: 930] loss: 44.76420974731445
[step: 931] loss: 43.97689437866211
[step: 932] loss: 43.44645690917969
[step: 933] loss: 43.649105072021484
[step: 934] loss: 44.08007049560547
[step: 935] loss: 44.539955139160156
[step: 936] loss: 43.431541442871094
[step: 937] loss: 43.5176887512207
[step: 938] loss: 44.026432037353516
[step: 939] loss: 43.31646728515625
[step: 940] loss: 43.11132049560547
[step: 941] loss: 43.56571578979492
[step: 942] loss: 43.42

In [10]:
len(answer)

28

In [11]:
answer

[31919.295046721436,
 28023.896400626025,
 24375.627162087185,
 20950.305165728831,
 18090.689262586988,
 16492.185803541714,
 16361.022539365167,
 17077.627250807684,
 17527.163219077185,
 17181.349378622759,
 16662.440893279501,
 16751.377424575498,
 17390.167397856108,
 17771.561147530927,
 17368.778954001489,
 16645.151323068123,
 16321.950961684437,
 16471.21827486117,
 16625.507399831353,
 16620.610971159385,
 16998.210406788887,
 18293.535998663916,
 20105.266830484994,
 21171.956902364105,
 21166.428423350382,
 21814.131936555455,
 25298.540713776478,
 31872.984188345559]

In [12]:
len(rawArrayDatas[1])

143